In [1]:
# !pip install evaluate
# import sys
# sys.path.append('/kaggle/input/qwertttt')

import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

import torch

import transformers
from transformers import BertTokenizer
from typing import *

from MEOW_Models.MT_models import MEOW_MTM
from MEOW_Models.Kernel_model import BertWithoutEmbedding

from MEOW_Utils.Data_utils import*
from MEOW_Utils.Training_utils import*

DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
BATCH_SIZE = 8
EPOCH_NUM = 10

ORG_FILE_PATH_CoLA = r'Dataset_infile\CoLA_Prompt.csv'
ORG_FILE_PATH_MNLI = r'Dataset_infile\MNLI.csv'
ORG_FILE_PATH_SQuAD = r'Dataset_infile\SQuAD.csv'

INPUT_FILE_PATH_CoLA = r'Dataset_infile\_CoLA.csv'
INPUT_FILE_PATH_MNLI = r'Dataset_infile\_MNLI.csv'
INPUT_FILE_PATH_SQuAD = r'Dataset_infile\_SQuAD.csv'
# INPUT_FILE_PATH_SQuAD = r'/kaggle/input/qwertttt/Data_file/SQuAD.csv'

SQuAD_DATASIZE = 30000
CoLA_DATASIZE = 8550  # max 8550
MNLI_DATASIZE = 15000  # max 15000

TEST_SIZE = 0.3
PRETRAINED_MODULE_NAME = 'bert-base-uncased'

In [3]:
tokenizer = BertTokenizer.from_pretrained(PRETRAINED_MODULE_NAME)
## use it if you only want to change the datasize of some dataset(dataframe)
# create_CoLA_df(ORG_FILE_PATH_CoLA, tokenizer, data_size=CoLA_DATASIZE)
# create_MNLI_df(ORG_FILE_PATH_MNLI, tokenizer, data_size=MNLI_DATASIZE)
# create_SQuAD_df(ORG_FILE_PATH_SQuAD, tokenizer, SQuAD_DATASIZE)

c:\Users\Administrator\codeblocks_workspace\MEOW\MEOW_Utils\Data_utils.py:167: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train['TKstart'][i], df_train['TKend'][i] = count_the_TKbeg_and_TKend(df_train.iloc[i]['context'], df_train.iloc[i]['answer_start'], df_train.iloc[i]['text'], tokenizer)


,index,question,context,text,answer_start,label,TKstart,TKend,SEP_ind
0,56be85543aeaaa14008c9063,When did Beyonce start becoming popular?,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,in the late 1990s,269,1,67,70,165
1,56be85543aeaaa14008c9065,What areas did Beyonce compete in when she was...,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,singing and dancing,207,1,55,57,165
2,56be85543aeaaa14008c9066,When did Beyonce leave Destiny's Child and bec...,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,2003,526,1,128,128,165
3,56bf6b0f3aeaaa14008c9601,In what city and state did Beyonce grow up?,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,"houston, texas",166,1,47,49,165
4,56bf6b0f3aeaaa14008c9602,In which decade did Beyonce become famous?,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,late 1990s,276,1,69,70,165
...,...,...,...,...,...,...,...,...,...
29971,57065bea75f01819005e7b82,The continuous back and forth regarding the de...,"In September 2006, German officials seized MP3...",bringing the patent wild west to germany,365,1,75,81,90
29972,57065e2352bb8914006899f0,"Who sued Apple, Samsung Electronics and Sandis...","In February 2007, Texas MP3 Technologies sued ...",texas mp3 technologies,18,1,5,7,56
29973,57065e2352bb8914006899f1,Which court did the lawsuit take place in?,"In February 2007, Texas MP3 Technologies sued ...",eastern texas federal court,88,1,17,20,56
29974,57065e2352bb8914006899f2,What was the claim that the lawsuit was based on?,"In February 2007, Texas MP3 Technologies sued ...",infringement of a portable mp3 player patent,126,1,23,29,56


In [5]:
#處理好 dataframe
df_CoLA = pd.read_csv(INPUT_FILE_PATH_CoLA, index_col=[0])
df_MNLI = pd.read_csv(INPUT_FILE_PATH_MNLI, index_col=[0])
df_SQuAD = pd.read_csv(INPUT_FILE_PATH_SQuAD, index_col=[0])

df_SQuAD_HA = df_SQuAD[df_SQuAD.answer_start != -1]
df_SQuAD_NA = df_SQuAD[df_SQuAD.answer_start == -1]

df_SQuAD.sample(n=8)

,index,question,context,text,answer_start,label,TKstart,TKend,SEP_ind
15942,56e104dfcd28a01900c67447,Where was the Zond 4 over when it was destroye...,The Soviet Zond spacecraft was not yet ready f...,gulf of guinea,558,1,131,133,173
25742,56f8ecd49e9bad19000a0709,Hogarth makes no mention of what?,Hogarth then proceeds to say where and why in ...,the classics,90,1,21,22,118
4012,56d10a4717492d1400aab7d8,What technological development led resulted in...,"New York City has more than 2,000 arts and cul...",electric lighting,440,1,81,82,158
27418,56fae59a8f12f31900630266,In what year was the SFA founded?,Growing out of the Somali people's rich storyt...,1975,264,1,49,49,246
20374,56e7b66337bdd419002c43c9,What are the two most prominent theatres in Na...,Most of Nanjing's major theatres are multi-pur...,the people's convention hall and the nanjing a...,165,1,32,44,63
22782,56f8aa0d9b226e1400dd0d97,About how much of UK's passenger traffic did S...,"Southampton has always been a port, and the do...",half,254,1,57,57,232
24134,56f8542caef2371900625fe4,Did number of cat of arms in the late middle a...,"The most notable difference is that, contrary ...",low and did not exceed 200,596,1,115,120,136
25839,56f8ff3d9b226e1400dd1244,Who works closely with the definition of the N...,Working closely in conjunction with the defini...,the near east south asia center for strategic ...,104,1,18,30,151


In [6]:
Helper = layer_helper(pretrained_module_name=PRETRAINED_MODULE_NAME, device=DEVICE)

SQuAD_NA_databox = DataBox(
                 dataset_name = 'SQuAD',
                 embedding_layer = Helper.get_embedding_layer(individual=True),
                 modeling_layer = Helper.get_modelings_layer_clf(individual_pooler=True),
                 df_Data = df_SQuAD_NA,
                 test_size = TEST_SIZE,
                 tokenizer = tokenizer,
                 label_nums = 2,
                 batch_size = BATCH_SIZE,
                 )

SQuAD_HA_databox = DataBox(
                 dataset_name = 'SQuAD',
                 embedding_layer = SQuAD_NA_databox.embedding_layer,
                 modeling_layer = SQuAD_NA_databox.modeling_layer,
                 df_Data = df_SQuAD_HA,
                 test_size = TEST_SIZE,
                 tokenizer = tokenizer,
                 label_nums = 2,
                 batch_size = BATCH_SIZE,
                 )

CoLA_databox = DataBox(
             dataset_name = 'CoLA',
             embedding_layer = Helper.get_embedding_layer(individual=True),
             modeling_layer = Helper.get_modelings_layer_clf(individual_pooler=True),
             df_Data = df_CoLA,
             test_size = TEST_SIZE,
             tokenizer = tokenizer,
             label_nums = 2,
             batch_size = BATCH_SIZE
             )

MNLI_databox = DataBox(
             dataset_name = 'MNLI',
             embedding_layer = Helper.get_embedding_layer(individual=True),
             modeling_layer = Helper.get_modelings_layer_clf(individual_pooler=True),
             df_Data = df_MNLI,
             test_size = TEST_SIZE,
             tokenizer = tokenizer,
             label_nums = 3,
             batch_size = BATCH_SIZE
             )


for i in SQuAD_NA_databox.test_dataloader:
    0
for i in SQuAD_HA_databox.test_dataloader:
    0
for i in MNLI_databox.test_dataloader:
    0
for i in CoLA_databox.test_dataloader:
    0

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertWithoutEmbedding: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertWithoutEmbedding from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertWithoutEmbedding from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [5]:
MEOW_model = MEOW_MTM(Helper.get_kernel_model(),
                      modeling_layer_for_qa = Helper.get_modelings_layer_qa(),
                      CoLA_databox=CoLA_databox,
                      MNLI_databox=MNLI_databox,
                      SQuAD_databox=SQuAD_HA_databox, #pass HA or NA is same
                      device = DEVICE)

# H = {
#     "train_loss": [],
#     "train_acc": [],
#     "val_loss":[],
#     "val_acc": []
#     }

Training_a_epoc_data_num = min(len(SQuAD_HA_databox.training_dataset), 
                               len(SQuAD_NA_databox.training_dataset), 
                               len(MNLI_databox.training_dataset),
                               len(CoLA_databox.training_dataset))
Training_round = int((Training_a_epoc_data_num+BATCH_SIZE-1) / BATCH_SIZE)

Test_a_epoc_data_num = min(len(SQuAD_HA_databox.test_dataset), 
                           len(SQuAD_NA_databox.test_dataset),
                           len(MNLI_databox.test_dataset),
                           len(CoLA_databox.test_dataset))
Test_round = int((Test_a_epoc_data_num+BATCH_SIZE-1) / BATCH_SIZE)

print(f'Per epoc round\'s num is {Training_round}')
print(f'Per epoc round\'s num is {Test_round}')

Per epoc round's num is 154
Per epoc round's num is 66


In [6]:
count_F1_score(MEOW_model, df_SQuAD, tokenizer, DEVICE)

['in', 'the', 'late', '1990s']
[]

['singing', 'and', 'dancing']
[]

['2003']
[]

['houston', ',', 'texas']
[]

['late', '1990s']
[]

['destiny', "'", 's', 'child']
[]

['dangerously', 'in', 'love']
[]

['mathew', 'knowles']
[]

['late', '1990s']
[]

['lead', 'singer']
[]

['dangerously', 'in', 'love']
[]

['2003']
[]

['five']
[]

['lead', 'singer']
[]

['dangerously', 'in', 'love']
[]

['acting']
[]

['jay', 'z']
[]

['six']
[]

['dream', '##girl', '##s']
[]

['2010']
[]

['beyonce']
[]



KeyboardInterrupt: 

In [ ]:
# 訓練
for epoch in range(EPOCH_NUM):
    print("the {:d} iter :".format(epoch+1))

    #### training now
    ####--------------------------------------------------------------------------------------------------
    MEOW_model.train()

    SQuAD_HA_training_loss = 0.0
    SQuAD_NA_training_loss = 0.0
    CoLA_training_loss = 0.0
    MNLI_training_loss = 0.0

    SQuAD_HA_training_correct = 0
    SQuAD_NA_training_correct = 0
    CoLA_training_correct = 0
    MNLI_training_correct = 0

    SQuAD_HA_training_iter = iter(SQuAD_HA_databox.training_dataloader)
    SQuAD_NA_training_iter = iter(SQuAD_NA_databox.training_dataloader)
    CoLA_training_iter = iter(CoLA_databox.training_dataloader)
    MNLI_training_iter = iter(MNLI_databox.training_dataloader)

    for i in range(Training_round):
        # SQuAD has_answer
        # loss, prob, correct = QA_running(MEOW_model, SQuAD_HA_training_iter, DEVICE, 'SQuAD', do_optimize=True)
        # print("the SQuAD has answer loss is : ", loss.item())
        # SQuAD_HA_training_loss += loss.item()
        # SQuAD_HA_training_correct += correct

        # SQuAD no_answer
        # loss, prob, correct = QA_running(MEOW_model, SQuAD_NA_training_iter, DEVICE, 'SQuAD', do_optimize=True)
        # print("the SQuAD no answer loss is : ", loss.item())
        # SQuAD_NA_training_loss += loss.item()
        # SQuAD_NA_training_correct += correct

        # CoLA
        loss, prob, correct = Classifiaction_running(MEOW_model, CoLA_training_iter, DEVICE, 'CoLA', do_optimize=True)
        print("the CoLA loss is : ", loss.item())
        CoLA_training_loss += loss.item()
        CoLA_training_correct += correct

        # MNLI
        loss, prob, correct = Classifiaction_running(MEOW_model, MNLI_training_iter, DEVICE, 'MNLI', do_optimize=True)
        print("the Sentimnet loss is : ", loss.item())
        MNLI_training_loss += loss.item()
        MNLI_training_correct += correct
        
    
    SQuAD_HA_avg_loss = SQuAD_HA_training_loss / Training_round
    SQuAD_HA_databox.H['train_loss'].append(SQuAD_HA_avg_loss)

    SQuAD_NA_avg_loss = SQuAD_NA_training_loss / Training_round
    SQuAD_NA_databox.H['train_loss'].append(SQuAD_NA_avg_loss)

    CoLA_avg_loss = CoLA_training_loss / Training_round
    CoLA_databox.H['train_loss'].append(CoLA_avg_loss)

    MNLI_avg_loss = MNLI_training_loss / Training_round
    MNLI_databox.H['train_loss'].append(MNLI_avg_loss)

    # SQuAD_f1_score = count_F1_score(MEOW_model, SQuAD_NA_databox.df_train, tokenizer, DEVICE)

    print("SQuAD has answer train loss: {:.6f}, correct_rate: {:.4f}".format(SQuAD_HA_avg_loss, SQuAD_HA_training_correct/Training_a_epoc_data_num))
    print("SQuAD no answer train loss: {:.6f}, correct_rate: {:.4f}".format(SQuAD_NA_avg_loss, SQuAD_NA_training_correct/Training_a_epoc_data_num))
    print("CoLA train loss: {:.6f}, correct_rate: {:.4f}".format(CoLA_avg_loss, CoLA_training_correct/Training_a_epoc_data_num))
    print("MNLI train loss: {:.6f}, correct_rate: {:.4f}".format(MNLI_avg_loss, MNLI_training_correct/Training_a_epoc_data_num))
    ####--------------------------------------------------------------------------------------------------
    ####--------------------------------------------------------------------------------------------------


    #### test now
    ####--------------------------------------------------------------------------------------------------
    MEOW_model.eval()
    
    SQuAD_HA_test_loss = 0.0
    SQuAD_NA_test_loss = 0.0
    CoLA_test_loss = 0.0
    MNLI_test_loss = 0.0

    SQuAD_HA_test_correct = 0
    SQuAD_NA_test_correct = 0
    CoLA_test_correct = 0
    MNLI_test_correct = 0

    SQuAD_HA_test_iter = iter(SQuAD_HA_databox.test_dataloader)
    SQuAD_NA_test_iter = iter(SQuAD_NA_databox.test_dataloader)
    CoLA_test_iter = iter(CoLA_databox.test_dataloader)
    MNLI_test_iter = iter(MNLI_databox.test_dataloader)

    for i in range(Test_round):
        # SQuAD has_answer
        loss, prob, correct = QA_running(MEOW_model, SQuAD_HA_test_iter, DEVICE, 'SQuAD', do_optimize=False)
        SQuAD_HA_test_loss += loss.item()
        SQuAD_HA_test_correct += correct
        print(loss.item())

        # SQuAD no_answer
        loss, prob, correct = QA_running(MEOW_model, SQuAD_NA_test_iter, DEVICE, 'SQuAD', do_optimize=False)
        SQuAD_NA_test_loss += loss.item()
        SQuAD_NA_test_correct += correct
        print(loss.item())

        # CoLA
        loss, prob, correct = Classifiaction_running(MEOW_model, CoLA_test_iter, DEVICE, 'CoLA', do_optimize=False)
        CoLA_test_loss += loss.item()
        CoLA_test_correct += correct
        print(loss.item())

        # MNLI
        loss, prob, correct = Classifiaction_running(MEOW_model, MNLI_test_iter, DEVICE, 'MNLI', do_optimize=False)
        MNLI_test_loss += loss.item()
        MNLI_test_correct += correct
        print(loss.item())


    SQuAD_HA_avg_loss = SQuAD_HA_test_loss / Test_round
    SQuAD_HA_databox.H['test_loss'].append(SQuAD_HA_avg_loss)

    SQuAD_NA_avg_loss = SQuAD_NA_test_loss / Test_round
    SQuAD_NA_databox.H['test_loss'].append(SQuAD_NA_avg_loss)
    
    CoLA_avg_loss = CoLA_test_loss / Test_round
    CoLA_databox.H['test_loss'].append(CoLA_avg_loss)

    MNLI_avg_loss = MNLI_test_loss / Test_round
    MNLI_databox.H['test_loss'].append(MNLI_avg_loss)
    
    # print("SQuAD test loss: {:.6f}, SQuAD f1 score: {:.4f}".format(SQuAD_avg_loss, SQuAD_f1_score))
    print("SQuAD has answer test loss: {:.6f}, correct_rate: {:.4f}".format(SQuAD_HA_avg_loss, SQuAD_HA_test_correct/Test_a_epoc_data_num))
    print("SQuAD no answer test loss: {:.6f}, correct_rate: {:.4f}".format(SQuAD_NA_avg_loss, SQuAD_NA_test_correct/Test_a_epoc_data_num))
    print("CoLA test loss: {:.6f}, correct_rate: {:.4f}".format(CoLA_avg_loss, CoLA_training_correct/Training_a_epoc_data_num))
    print("MNLI test loss: {:.6f}, correct_rate: {:.4f}".format(MNLI_avg_loss, MNLI_training_correct/Training_a_epoc_data_num))
    print('')
    ####--------------------------------------------------------------------------------------------------
    ####--------------------------------------------------------------------------------------------------